In [2]:
from keras import layers
from keras import models
from keras import losses
from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/home/yaagocruuz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Questão 1 - Implementar uma MLP

In [3]:
import numpy as np
import sys


class NeuralNetMLP(object):  
    #vetor de pesos
    #       w1 w2 w3  w4 w5 w6  w7 w8 w9
    #pesos = [2, 1, 1, -2, 1, 2, -1, 3, 2]
    
    def __init__(self, n_hidden=30,
                 l2=0., epochs=100, eta=0.001,
                 shuffle=True, minibatch_size=1, seed=None):

        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size

    def _onehot(self, y, n_classes):
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T
    

    def _ReLU(self, z):
            return np.maximum(0,z)
    
    def _ReLUderivada(self, z):
        if (z > 0):
            return 1
        else:
            return 0
    
    def _forward(self, X):
        z_h = np.dot(X, self.w_h)
        #hidden layer ativacao
        a_h = self._ReLU(z_h)
        #output
        z_out = np.dot(a_h, self.w_out)
        #output layer ativacao
        a_out = self._ReLU(z_out)

        return z_h, a_h, z_out, a_out

    def _compute_cost(self, y_enc, output):

        L2_term = (self.l2 *
                   (np.sum(self.w_h ** 2.) +
                    np.sum(self.w_out ** 2.)))

        term1 = -y_enc * (np.log(output))
        term2 = (1. - y_enc) * np.log(1. - output)
        
        cost = np.sum(term1 - term2) + L2_term
        return cost

    def predict(self, X):
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train):
        n_output = np.unique(y_train).shape[0]  # number of class labels
        n_features = X_train.shape[1]

        ########################
        # Weight initialization
        ########################
        
        
        # weights for input -> hidden
        
        #self.b_h = np.zeros(self.n_hidden)
        self.w_h =  np.array([[2, 1 , 1], [1, -2, 2]])
        self.w_h = self.w_h.astype(np.float64)
        #self.w_h = self.random.normal(loc=0.0, scale=0.1,
        #                             size=(n_features, self.n_hidden))
        
        # weights for hidden -> output
        #self.b_out = np.zeros(n_output)
        self.w_out =  np.array([[-1], [3], [2]])
        self.w_out = self.w_out.astype(np.float64)
        #self.w_out = self.random.normal(loc=0.0, scale=0.1,
        #                                size=(self.n_hidden, n_output))

        epoch_strlen = len(str(self.epochs))  # for progress formatting
        self.eval_ = {'cost': [], 'train_acc': [], 'valid_acc': []}

        y_train_enc = self._onehot(y_train, n_output)

        # iterate over training epochs
        for i in range(self.epochs):

            # iterate over minibatches
            indices = np.arange(X_train.shape[0])

            if self.shuffle:
                self.random.shuffle(indices)

            for start_idx in range(0, indices.shape[0] - self.minibatch_size +
                                   1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]

                # forward propagation
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])

                ##################
                # Backpropagation
                ##################

                # [n_samples, n_classlabels]
                sigma_out = a_out - y_train_enc[batch_idx]

                # [n_samples, n_hidden]
                #derivada de relu é 
                sigmoid_derivative_h = a_h
                
                # [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
                # -> [n_samples, n_hidden]
                sigma_h = (np.dot(sigma_out, self.w_out.T) *
                           sigmoid_derivative_h)

                # [n_features, n_samples] dot [n_samples, n_hidden]
                # -> [n_features, n_hidden]
                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)

                # [n_hidden, n_samples] dot [n_samples, n_classlabels]
                # -> [n_hidden, n_classlabels]
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)

                # Regularization and weight updates
                delta_w_h = (grad_w_h + self.l2*self.w_h)
                self.w_h -= self.eta * delta_w_h

                delta_w_out = (grad_w_out + self.l2*self.w_out)
                self.w_out -= self.eta * delta_w_out

            #############
            # Evaluation
            #############

            # Evaluation after each epoch during training
            z_h, a_h, z_out, a_out = self._forward(X_train)
            
            cost = self._compute_cost(y_enc=y_train_enc,
                                      output=a_out)
            
            y_train_pred = self.predict(X_train)
            
            train_acc = ((np.sum(y_train == y_train_pred)).astype(np.float) /
                         X_train.shape[0])
            
            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)

        return self

In [4]:
#valores de x1 e x2
X = np.array([[1, 2]])

#resultado
y = np.array([3])
y = np.where(y == 3, 0, 1)

In [7]:
epoch = [10, 50, 100]
lr = [0.1, 0.01, 0.001]

for ep in epoch:
    for l in lr:
        mlp = NeuralNetMLP(epochs=ep, eta=l)
        mlp.fit(X, y)
        print('lr= {} , epoch= {}'.format(l, ep))
        print('w_h:\n{}\n'.format(mlp.w_h))
        print('w_out:\n{}\n'.format(mlp.w_out))

lr= 0.1 , epoch= 10
w_h:
[[-0.2  1.  -4. ]
 [-3.4 -2.  -8. ]]

w_out:
[[-1.6]
 [ 3. ]
 [-0.5]]

lr= 0.01 , epoch= 10
w_h:
[[ 2.05979713  1.          0.62046002]
 [ 1.11959425 -2.          1.24092005]]

w_out:
[[-1.07930561]
 [ 3.        ]
 [ 1.81797992]]

lr= 0.001 , epoch= 10
w_h:
[[ 2.12014496  1.          0.76664092]
 [ 1.24028991 -2.          1.53328185]]

w_out:
[[-1.11436199]
 [ 3.        ]
 [ 1.88019007]]

lr= 0.1 , epoch= 50
w_h:
[[-0.2  1.  -4. ]
 [-3.4 -2.  -8. ]]

w_out:
[[-1.6]
 [ 3. ]
 [-0.5]]

lr= 0.01 , epoch= 50
w_h:
[[ 2.05979713  1.          0.62046002]
 [ 1.11959425 -2.          1.24092005]]

w_out:
[[-1.07930561]
 [ 3.        ]
 [ 1.81797992]]

lr= 0.001 , epoch= 50
w_h:
[[ 2.16409983  1.          0.70966308]
 [ 1.32819965 -2.          1.41932616]]

w_out:
[[-1.1531765 ]
 [ 3.        ]
 [ 1.84965566]]

lr= 0.1 , epoch= 100
w_h:
[[-0.2  1.  -4. ]
 [-3.4 -2.  -8. ]]

w_out:
[[-1.6]
 [ 3. ]
 [-0.5]]

lr= 0.01 , epoch= 100
w_h:
[[ 2.05979713  1.          0.62046002]
 [ 

## há pouca variação de pesos não importando o learning rate e as épocas.